# Imports

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import matplotlib.pyplot as plt
import random

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.classification import F1Score

from dgl.nn.pytorch import GraphConv
from ray import tune

# Starting with the gene data

Building the graph and getting gene names, ids and if they are pathogenic or not.

In [ ]:
links = []
with open("Data\human_links.tsv") as file: # the dataset containing genes and the genes they interact with
    tsv_file = csv.reader(file, delimiter="\t")
    for line in tsv_file:
        links.append(line)

links = np.asarray(links)
links

In [ ]:
links_df = pd.DataFrame(links, columns =['gene1', 'gene2'])
links_df
# read: there is an interaction between gene1 and gene2

In [ ]:
G = nx.Graph()
G.add_nodes_from([]) # number of nodes added = number of unique genes in links df that is added below.
G.add_edges_from(np.asarray(links_df)) # add the connections between the nodes

In [ ]:
nodes_ss = list(G.nodes) # all of the genes in the graph - will be used later to filter
nodes_ss

In [ ]:
# the data that tells us if a gene is pathogenic or not

gene_data_non_path = pd.read_excel("Data\interactome_vs_mitocarta.xlsx", sheet_name="non-pathogenic", names=["gene"])
gene_data_non_path["pathogenic"] = 0

gene_data_path = pd.read_excel("Data\interactome_vs_mitocarta.xlsx", sheet_name="pathogenic_in_interactome", names=["gene"])
gene_data_path["pathogenic"] = 1

# concat the pathogenic and non-pathogenic genes
gene_data = pd.concat([gene_data_non_path, gene_data_path])

# shuffle the genes so that the pathogenic and non-pathogenic genes are mixed
gene_data = gene_data.sample(frac=1, random_state=207).reset_index().drop(["index"], axis=1)
gene_data

In [ ]:
# filter the genes and keep only the genes that are in the graph
gene_data_ss = gene_data.loc[gene_data['gene'].isin(nodes_ss)]
gene_data_ss

In [ ]:
# get the HGNC symbols and ENSG id for all the genes to merge the datasets later
gene_functions = pd.read_csv("Data\human_gene_function.txt", delimiter="\t")
gene_functions = gene_functions[["HGNC_symbol", "ENSG_ID"]]
gene_functions

In [ ]:
# get the HGNC symbol for all the genes
gene_data_ss = gene_data_ss.loc[gene_data_ss['gene'].isin(gene_functions["HGNC_symbol"])]
gene_data_ss = gene_data_ss.merge(gene_functions, left_on="gene", right_on="HGNC_symbol").drop("HGNC_symbol", axis=1)
gene_data_ss

## Transcriptomics

In [ ]:
transcriptomics = pd.read_csv("Data\smirnov_transcriptomics.csv")
transcriptomics

In [ ]:
# some string comprehension to get the "base" ENSG id
transcriptomics["gene_id"] = transcriptomics["gene_id"].str.split('\.').str[0].str.strip()
transcriptomics

In [ ]:
transcriptomics = transcriptomics.rename({'gene_id': 'geneID'}, axis=1)

# I create a separate dataset specifically for CCA and PLS, since the structure is different
# merge, drop duplicate columns, and rename columns
all_data_tr_weights_analysis = pd.merge(gene_data_ss, transcriptomics, left_on="ENSG_ID", right_on="geneID", how="inner")
all_data_tr_weights_analysis = all_data_tr_weights_analysis.drop(["ENSG_ID", "geneID"], axis=1)
all_data_tr_weights_analysis = all_data_tr_weights_analysis.rename({'gene': 'geneID'}, axis=1)
all_data_tr_weights_analysis

## Proteomics

In [ ]:
proteomics = pd.read_csv("Data\smirnov_proteomics.tsv", delimiter="\t")
proteomics

In [ ]:
# keep the patients that are in both datasets
all_data_pr_weights_analysis = proteomics[proteomics.columns.intersection(all_data_tr_weights_analysis.columns)]
all_data_pr_weights_analysis

In [ ]:
all_data_tr_weights_analysis = all_data_tr_weights_analysis[all_data_tr_weights_analysis.columns.intersection(proteomics.columns)]
all_data_tr_weights_analysis

Back to the data for the graphs:

In [ ]:
all_data_tr = pd.merge(gene_data_ss, transcriptomics, left_on="ENSG_ID", right_on="geneID", how="inner")
all_data_tr = all_data_tr.drop(["ENSG_ID", "geneID"], axis=1)
all_data_tr = all_data_tr.set_index(["gene", "pathogenic"]).add_prefix("tr__").reset_index() # add prefix to identify transcriptomic
all_data_tr

In [ ]:
all_data_pr = pd.merge(gene_data_ss, proteomics, left_on="gene", right_on="geneID", how="inner")
all_data_pr = all_data_pr.drop(["ENSG_ID", "geneID"], axis=1)
all_data_pr = all_data_pr.set_index(['gene', 'pathogenic']).add_prefix('pr__').reset_index() # add prefix to identify proteomic
all_data_pr

In [ ]:
# scale and transform proteomics and transcriptomics separately before merging
scaler = MinMaxScaler()
transformer = QuantileTransformer(n_quantiles=100, output_distribution='normal')

all_data_tr.iloc[:, 2:] = scaler.fit_transform(transformer.fit_transform(all_data_tr.iloc[:, 2:]))
all_data_pr.iloc[:, 2:] = scaler.fit_transform(transformer.fit_transform(all_data_pr.iloc[:, 2:]))

## The final clean data
With proteomics and transcriptomics merged together:

In [ ]:
# We now have: Rows = Genes, Columns = each patient's transcriptomic or proteomic RNA-seq count per gene
all_data = pd.merge(all_data_tr, all_data_pr, on=["gene", "pathogenic"], how="inner")
all_data

# Check for duplicates

In [ ]:
idx = all_data['gene'][all_data['gene'].duplicated(keep=False)]
idx

In [ ]:
list(idx.index)[1]

In [ ]:
# remove the duplicated gene
all_data = all_data.drop(list(idx.index)[1], axis=0)
all_data

Finally, there are 716 genes and values for 230 proteomic and 146 transcriptomic patients. To build the graph the difference doesn't matter since each gene gets stored in an array in each node - no need for transcriptomics = proteomics.

# Creating the final graph with the 716 genes

In [ ]:
# we create one graph for each "weights analysis": The none, CCA, PLS, and pearson analyses
G_none = nx.subgraph(G, all_data["gene"])
G_cca = nx.subgraph(G, all_data["gene"])
G_pls = nx.subgraph(G, all_data["gene"])
G_pearson = nx.subgraph(G, all_data["gene"])

In [ ]:
# The nodes need to be labelled numerically for the DGL library, so we do this
relabel_nodes = dict((v,k) for k,v in all_data.iloc[:,0].to_dict().items())

In [ ]:
G_none = nx.relabel_nodes(G_none, relabel_nodes, copy=True)
G_cca = nx.relabel_nodes(G_cca, relabel_nodes, copy=True)
G_pls = nx.relabel_nodes(G_pls, relabel_nodes, copy=True)
G_pearson = nx.relabel_nodes(G_pearson, relabel_nodes, copy=True)

# Adding weights

## Canonical Correlation Analysis for weights

In [ ]:
# we need to find the correlations between the genes per omic, so the genes need to become the columns -> transpose datasets
all_data_tr_cca_transposed = all_data_tr_weights_analysis.T
all_data_tr_cca_transposed.columns = all_data_tr_cca_transposed.iloc[0] #set row 0 as column names
all_data_tr_cca_transposed = all_data_tr_cca_transposed.drop(["geneID"], axis=0) # remove row with pathogenic label
all_data_tr_cca_transposed

In [ ]:
all_data_pr_cca_transposed = all_data_pr_weights_analysis.T
all_data_pr_cca_transposed.columns = all_data_pr_cca_transposed.iloc[0] #set row 0 as column names
all_data_pr_cca_transposed = all_data_pr_cca_transposed.drop(["geneID"], axis=0) # remove row with pathogenic label
all_data_pr_cca_transposed

In [ ]:
# CCA
X1 = all_data_pr_cca_transposed
X2 = all_data_tr_cca_transposed

# same scaling and transforming steps as previously
scaler = MinMaxScaler()
transformer = QuantileTransformer(n_quantiles=100, output_distribution='normal')

X1_sc = scaler.fit_transform(transformer.fit_transform(X1))
X2_sc = scaler.fit_transform(transformer.fit_transform(X2))

n_comp=1 #choose number of canonical variates pairs 
cca = CCA(scale=False, n_components=n_comp) #define CCA
cca.fit(X1_sc, X2_sc) #fit scaled data
X1_c, X2_c = cca.transform(X1_sc, X2_sc) #transform our datasests to obtain canonical variates

In [ ]:
# get analysis results into correlation df
coef_df = pd.DataFrame(np.round(cca.coef_, 6), columns = [X2.columns])
coef_df.index = X1.columns
coef_df

In [ ]:
# check the heatmap of the first 30 genes
import seaborn as sns
sns.heatmap(coef_df.iloc[0:30, 0:30], cmap='coolwarm', annot=False, linewidths=1, vmin=-0.032)

In [ ]:
def melt_and_clean(coef_df):
    # get coefficients to use as weights
    coef_df_melted = pd.melt(coef_df, id_vars=None, value_vars=None, var_name='column', value_name='value', 
                             ignore_index = False).reset_index()
    coef_df_melted = coef_df_melted.rename({"geneID":"node1", "column":"node2", "value":"weight"}, axis=1)

    # remove duplicates
    coef_df_melted = coef_df_melted.drop_duplicates(subset=['node1', 'node2'], keep='first')
    return(coef_df_melted)

In [ ]:
coef_df_melted_cca = melt_and_clean(coef_df)
coef_df_melted_cca

In [ ]:
# keep only the genes that we have in our links df
cca_graph_genes = pd.merge(links_df, coef_df_melted_cca, left_on=["gene1", "gene2"], right_on=["node1", "node2"], how="inner")
cca_graph_genes = cca_graph_genes.drop(["gene1", "gene2"], axis=1)
cca_graph_genes

In [ ]:
def get_graph_weights(graph_genes, neg_weights):
    # keep only the genes that is that is in the final data
    graph_genes_ = graph_genes[graph_genes['node1'].isin(all_data['gene'])]
    graph_genes_ = graph_genes_[graph_genes_["node2"].isin(all_data['gene'])]
    if neg_weights == False:
        graph_genes_["weight"] = np.abs(graph_genes_["weight"])
    return(graph_genes_)

In [ ]:
def map_node_names_to_numeric(graph_genes_):
    # map node names to numerical node numbers for DGL
    graph_genes_['n1'] = graph_genes_['node1'].map(lambda x: all_data.set_index('gene').index.get_loc(x))
    graph_genes_['n2'] = graph_genes_['node2'].map(lambda x: all_data.set_index('gene').index.get_loc(x))
    return(graph_genes_)

In [ ]:
def get_valid_edges(graph_genes, G_, neg_weights):
    
    graph_genes_ = get_graph_weights(graph_genes, neg_weights)
    graph_genes_ = map_node_names_to_numeric(graph_genes_) # takes a while
    #display(graph_genes_)
    
    # create a dict with a tuple of the interacting genes as the key and the weight as value - this is the format needed to add
    # it to the graph
    genes_dict = graph_genes_.iloc[:,2:].set_index(["n1", "n2"]).to_dict(orient='index')
    
    nx.set_edge_attributes(G_, genes_dict, "name")
    # only the edges that exist in the graph now have a weight, so we need to extract these "valid" edges
    
    valid_edges = nx.get_edge_attributes(G_, "name")
    #print(valid_edges)
    # create df of valid edges
    valid_analysis_edges = pd.DataFrame.from_dict(valid_edges).T.reset_index()
    valid_analysis_edges = valid_analysis_edges.rename({'level_0': 'node1', "level_1":"node2"}, axis=1)
    return(valid_analysis_edges)

In [ ]:
valid_cca_edges = get_valid_edges(cca_graph_genes, G_cca, neg_weights=True)
valid_cca_edges

# PLS Regression for determining weights

In [ ]:
n_comp=1 
PLSR = PLSRegression(scale=False, n_components=n_comp) 
PLSR.fit(X1_sc, X2_sc) 
X1_c, X2_c = PLSR.transform(X1_sc, X2_sc) 

In [ ]:
# get analysis results into correlation df
coef_df_pls = pd.DataFrame(np.round(PLSR.coef_, 6), columns = [X2.columns])
coef_df_pls.index = X1.columns
coef_df_pls

In [ ]:
# check the heatmap of the first 30 genes
import seaborn as sns
sns.heatmap(coef_df_pls.iloc[0:30, 0:30], cmap='coolwarm', annot=False, linewidths=1, vmin=-0.00025)

In [ ]:
coef_df_melted_pls = melt_and_clean(coef_df_pls)
coef_df_melted_pls

In [ ]:
# keep only the genes in the graph
pls_graph_genes = pd.merge(links_df, coef_df_melted_pls, left_on=["gene1", "gene2"], right_on=["node1", "node2"], how="inner")
pls_graph_genes = pls_graph_genes.drop(["gene1", "gene2"], axis=1)
pls_graph_genes

In [ ]:
valid_pls_edges = get_valid_edges(pls_graph_genes, G_pls, neg_weights=True)
valid_pls_edges

# Pearson's cross correlation

In [ ]:
# need the same column names for transcriptomics and proteomics
pearson_keep_columns = list(set(all_data_pr_cca_transposed.columns) & set(all_data_tr_cca_transposed.columns))
all_data_pr_pc_transposed = all_data_pr_cca_transposed[pearson_keep_columns]
all_data_tr_pc_transposed = all_data_tr_cca_transposed[pearson_keep_columns]

In [ ]:
# remove duplicated column
all_data_tr_pc_transposed = all_data_tr_pc_transposed.loc[:,~all_data_tr_pc_transposed.columns.duplicated()].copy()

In [ ]:
corr_matrix = pd.DataFrame(np.corrcoef(all_data_pr_pc_transposed.T.values.astype(float), 
                                       all_data_tr_pc_transposed.T.values.astype(float)))
corr_matrix

In [ ]:
#To get the cross correlation we need to take either the upper right or lower left of the matrix
# --                                     --
# | correlation         cross correlation |
# | cross correlation   correlation       |
# --                                     --

# taking upper right
corr_matrix = corr_matrix.iloc[:716, 716:]

In [ ]:
# rename columns and rows to genes
corr_matrix = corr_matrix.rename(columns={old: new for old, new in zip(corr_matrix.columns, all_data_pr_pc_transposed.columns)})
corr_matrix.index = all_data_pr_pc_transposed.columns
corr_matrix

In [ ]:
coef_df_melted_pearson = melt_and_clean(corr_matrix)
coef_df_melted_pearson

In [ ]:
# keep only the genes in the graph
pearson_graph_genes = pd.merge(links_df, coef_df_melted_pearson, left_on=["gene1", "gene2"], right_on=["node1", "node2"], 
                               how="inner")
pearson_graph_genes = pearson_graph_genes.drop(["gene1", "gene2"], axis=1)
pearson_graph_genes

In [ ]:
valid_pearson_edges = get_valid_edges(pearson_graph_genes, G_pearson, neg_weights=True)
valid_pearson_edges

# Building the DGL graphs

In [ ]:
def build_dgl_network(graph_weights_method, edge_df, weight_method):
    G_dgl = dgl.from_networkx(graph_weights_method) # transform to dgl graph
    G_dgl.ndata['feat'] = torch.from_numpy(all_data.iloc[:, 2:].values) # add omics data to nodes
  
    if weight_method != "None":
        # Create edge weights
        edge_weights = np.array(edge_df["weight"])
      
        # Add edges with weights
        src = np.array(edge_df["node1"])
        dst = np.array(edge_df["node2"])
        edge_weights = torch.tensor(edge_weights)
      
        G_dgl.add_edges(torch.tensor(src), torch.tensor(dst), data={'weight': edge_weights})
  
    return(G_dgl)

In [ ]:
G_dgl_none = build_dgl_network(G_none, valid_pearson_edges, "None") # use any edge data as placeholder - will not be added
G_dgl_none

In [ ]:
# can uncomment below and change neg_weights to change graph weights
#valid_cca_edges = get_valid_edges(cca_graph_genes, G_cca, neg_weights=True)
G_dgl_cca = build_dgl_network(G_cca, valid_cca_edges, "CCA")
G_dgl_cca

In [ ]:
# can uncomment below and change neg_weights to change graph weights
#valid_pls_edges = get_valid_edges(pls_graph_genes, G_pls, neg_weights=True)
G_dgl_pls = build_dgl_network(G_pls, valid_pls_edges, "PLS")
G_dgl_pls

In [ ]:
# can uncomment below and change neg_weights to change graph weights
#valid_pearson_edges = get_valid_edges(pearson_graph_genes, G_pearson, neg_weights=True)
G_dgl_pearson = build_dgl_network(G_pearson, valid_pearson_edges, "PEARSON")
G_dgl_pearson

# Building the GCN:

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes, weight = torch.double):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size, weight=weight)
        self.conv2 = GraphConv(hidden_size, hidden_size, weight=weight)
        self.conv3 = GraphConv(hidden_size, hidden_size, weight=weight)
        self.conv4 = GraphConv(hidden_size, num_classes, weight=weight)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = self.conv2(g, h)
        h = torch.relu(h)
        h = self.conv3(g, h)
        h = torch.relu(h)
        h = self.conv4(g, h)
        return h

In [ ]:
def get_labeled_nodes(node_labels): 
    # Count the number of 0s and 1s
    count_0 = np.count_nonzero(node_labels == 0)
    count_1 = np.count_nonzero(node_labels == 1)
    
    num_to_select = 16
    
    # Create a list of indices of 0s and 1s
    indices_0 = np.argwhere(node_labels == 0).flatten()
    indices_1 = np.argwhere(node_labels == 1).flatten()
    
    # Randomly select indices
    random.seed(207)
    selected_indices_0 = random.sample(list(indices_0), num_to_select)
    random.seed(702)
    selected_indices_1 = random.sample(list(indices_1), num_to_select)
    
    final_indices = np.concatenate((selected_indices_0, selected_indices_1))
    
    # get unlabelled indices
    unselected_indices = np.setdiff1d(np.arange(len(node_labels)), final_indices)
    
    return(final_indices, unselected_indices)

In [ ]:
node_labels = all_data['pathogenic'].astype('category').cat.codes.to_numpy()
random_labels, random_unlabelled = get_labeled_nodes(node_labels)
node_labels = torch.from_numpy(node_labels)

In [ ]:
weight_methods = ["No weights", "CCA", "PLS", "Pearson"]
i = 0
input_shape = all_data.iloc[:, 2:].shape[1]
hidden_size = 50
learning_rate = 0.0001

for g in [G_dgl_none, G_dgl_cca, G_dgl_pls, G_dgl_pearson]:
  
    print("Graph:", weight_methods[i])
  
    # build network
    net = GCN(input_shape, hidden_size, 2, weight=torch.double)
  
    inputs = g.ndata['feat'].double()
    labeled_nodes = torch.tensor(random_labels).long()
    g.ndata['label'] = node_labels
    labels = g.ndata['label']
    unlabeled_nodes = torch.tensor(random_unlabelled).long()
  
    g = dgl.add_self_loop(g)
  
    # save all evaluation metrics
    all_logits = []
    labeled_loss = []
    unlabeled_loss = []
    labeled_acc = []
    unlabeled_acc = []
    labeled_f1 = []
    unlabeled_f1 = []
    
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    f1 = F1Score(task="multiclass", num_classes=2, average="weighted")
    n_epochs = 1000
    
    for epoch in range(n_epochs):
        logits = net(g, inputs.float())
        # save the logits for visualization later
        all_logits.append(logits.detach())
        logp = F.log_softmax(logits, 1)
        
        # labeled loss
        loss = F.nll_loss(logp[labeled_nodes], labels[labeled_nodes].type(torch.LongTensor))
        labeled_loss.append(loss)
    
        # labeled accuracy
        _, predicted = torch.max(logp[labeled_nodes], 1)
        correct = (predicted == labels[labeled_nodes]).sum().item()
        accuracy = correct / len(labeled_nodes)
        labeled_acc.append(accuracy)
        
        # labeled f1 score
        f1_score = f1(predicted, labels[labeled_nodes])
        labeled_f1.append(f1_score)
        
        # unlabeled loss
        loss_unlabeled = F.nll_loss(logp[unlabeled_nodes], labels[unlabeled_nodes].type(torch.LongTensor))
        unlabeled_loss.append(loss_unlabeled)
    
        # unlabeled accuracy
        _, predicted = torch.max(logp[unlabeled_nodes], 1)
        correct = (predicted == labels[unlabeled_nodes]).sum().item()
        accuracy_unl = correct / len(unlabeled_nodes)
        unlabeled_acc.append(accuracy_unl)
        
        # unlabeled f1 score
        f1_score_unl = f1(predicted, labels[unlabeled_nodes])
        unlabeled_f1.append(f1_score_unl)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if epoch % 10 == 0:
            print('Epoch %d | Loss: %.4f  |  Loss Test: %.4f | Acc: %.4f  |  Acc Test: %.4f' % (epoch, loss, loss_unlabeled, accuracy, accuracy_unl))
  
    plt.plot(range(0, n_epochs), [loss.detach().numpy() for loss in labeled_loss], label="labeled")
    plt.plot(range(0, n_epochs), [loss.detach().numpy() for loss in unlabeled_loss], label="unlabeled")
    plt.legend()
    plt.title(f"Loss: {weight_methods[i]}")
    plt.show()
  
    plt.plot(range(0, n_epochs), [acc for acc in labeled_acc], label="labeled")
    plt.plot(range(0, n_epochs), [acc for acc in unlabeled_acc], label="unlabeled")
    plt.legend()
    plt.title(f"Accuracy: {weight_methods[i]}")
    plt.show()
    
    min_unl_loss = np.argmin([i.item() for i in unlabeled_loss])
    print(f"Lowest unlabelled loss: {min(unlabeled_loss).detach().item()} at epoch {min_unl_loss}, train loss: {[i.item() for i in labeled_loss][min_unl_loss]}")
    
    print(f"Maximum acc: {max(unlabeled_acc)} at epoch {min_unl_loss}, train acc: {labeled_acc[min_unl_loss]}")

    print(f"Maximum f1: {max(unlabeled_f1)} at epoch {min_unl_loss}, train f1: {labeled_f1[min_unl_loss]}")
  
    i += 1

# Hyper parameter tuning with Ray Tune

In [ ]:
def train_eval_gcn(config, reporter):
    input_shape = all_data.iloc[:, 2:].shape[1]
    
    g = config["g"]
    net = GCN(input_shape, config["hidden_neurons"], 2, weight=torch.double)
    
    inputs = g.ndata['feat'].double()
    labeled_nodes = torch.tensor(random_labels).long()
    g.ndata['label'] = node_labels
    labels = g.ndata['label']
    unlabeled_nodes = torch.tensor(random_unlabelled).long()
    
    g = dgl.add_self_loop(g)
    
    # save all evaluation metrics
    all_logits = []
    labeled_loss = []
    unlabeled_loss = []
    labeled_acc = []
    unlabeled_acc = []
    labeled_f1 = []
    unlabeled_f1 = []
    
    optimizer = torch.optim.Adam(net.parameters(), lr=config["learn_rate"])
    f1 = F1Score(task="multiclass", num_classes=2, average="weighted")
    n_epochs = 1000
    
    for epoch in range(n_epochs):
        logits = net(g, inputs.float())
        # save the logits for visualization later
        all_logits.append(logits.detach())
        logp = F.log_softmax(logits, 1)
        
        # labeled loss
        loss = F.nll_loss(logp[labeled_nodes], labels[labeled_nodes].type(torch.LongTensor))
        labeled_loss.append(loss)
    
        # labeled accuracy
        _, predicted = torch.max(logp[labeled_nodes], 1)
        correct = (predicted == labels[labeled_nodes]).sum().item()
        accuracy = correct / len(labeled_nodes)
        labeled_acc.append(accuracy)
        
        # labeled f1 score
        f1_score = f1(predicted, labels[labeled_nodes])
        labeled_f1.append(f1_score)
        
        # unlabeled loss
        loss_unlabeled = F.nll_loss(logp[unlabeled_nodes], labels[unlabeled_nodes].type(torch.LongTensor))
        unlabeled_loss.append(loss_unlabeled)
    
        # unlabeled accuracy
        _, predicted = torch.max(logp[unlabeled_nodes], 1)
        correct = (predicted == labels[unlabeled_nodes]).sum().item()
        accuracy_unl = correct / len(unlabeled_nodes)
        unlabeled_acc.append(accuracy_unl)
        
        # unlabeled f1 score
        f1_score_unl = f1(predicted, labels[unlabeled_nodes])
        unlabeled_f1.append(f1_score_unl)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    reporter({"test loss": min(unlabeled_loss).detach().item(), "train loss": [i.item() for i in labeled_loss][min_unl_loss], 
              "test acc": max(unlabeled_acc), "train acc": labeled_acc[min_unl_loss], 
              "test f1score": max(unlabeled_f1).item(), "train f1score": labeled_f1[min_unl_loss].item(), "epoch":min_unl_loss})


In [ ]:
# runs very long
weight_methods = ["No weights", "CCA", "PLS", "Pearson"]
i = 0

for g in [G_dgl_none, G_dgl_cca, G_dgl_pls, G_dgl_pearson]:   
    
    search_space = {
    "hidden_neurons": tune.grid_search([20, 30, 50, 80, 100, 120]),
    "learn_rate": tune.grid_search([0.01, 0.001, 0.0001, 0.00001]),
    "g": g,}
    
    print("Graph:", weight_methods[i])
    tuner = tune.Tuner(train_eval_gcn, param_space=search_space)
    results = tuner.fit()
    i += 1 

# Create an animation of the GCN's training

In [ ]:
weight_methods = ["CCA"]
i = 0
input_shape = all_data.iloc[:, 2:].shape[1]
hidden_size = 100
learning_rate = 0.0001

for g in [G_dgl_cca]:
  
    print("Graph:", weight_methods[i])
  
    # build network
    net = GCN(input_shape, hidden_size, 2, weight=torch.double)
  
    inputs = g.ndata['feat'].double()
    labeled_nodes = torch.tensor(random_labels).long()
    g.ndata['label'] = node_labels
    labels = g.ndata['label']
    unlabeled_nodes = torch.tensor(random_unlabelled).long()
  
    g = dgl.add_self_loop(g)
  
    # save all evaluation metrics
    all_logits = []
    labeled_loss = []
    unlabeled_loss = []
    labeled_acc = []
    unlabeled_acc = []
    labeled_f1 = []
    unlabeled_f1 = []
    
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    f1 = F1Score(task="multiclass", num_classes=2, average="weighted")
    n_epochs = 1000
    
    for epoch in range(n_epochs):
        logits = net(g, inputs.float())
        # save the logits for visualization later
        all_logits.append(logits.detach())
        logp = F.log_softmax(logits, 1)
        
        # labeled loss
        loss = F.nll_loss(logp[labeled_nodes], labels[labeled_nodes].type(torch.LongTensor))
        labeled_loss.append(loss)
    
        # labeled accuracy
        _, predicted = torch.max(logp[labeled_nodes], 1)
        correct = (predicted == labels[labeled_nodes]).sum().item()
        accuracy = correct / len(labeled_nodes)
        labeled_acc.append(accuracy)
        
        # labeled f1 score
        f1_score = f1(predicted, labels[labeled_nodes])
        labeled_f1.append(f1_score)
        
        # unlabeled loss
        loss_unlabeled = F.nll_loss(logp[unlabeled_nodes], labels[unlabeled_nodes].type(torch.LongTensor))
        unlabeled_loss.append(loss_unlabeled)
    
        # unlabeled accuracy
        _, predicted = torch.max(logp[unlabeled_nodes], 1)
        correct = (predicted == labels[unlabeled_nodes]).sum().item()
        accuracy_unl = correct / len(unlabeled_nodes)
        unlabeled_acc.append(accuracy_unl)
        
        # unlabeled f1 score
        f1_score_unl = f1(predicted, labels[unlabeled_nodes])
        unlabeled_f1.append(f1_score_unl)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if epoch % 10 == 0:
            print('Epoch %d | Loss: %.4f  |  Loss Test: %.4f | Acc: %.4f  |  Acc Test: %.4f' % (epoch, loss, loss_unlabeled, accuracy, accuracy_unl))
  
    plt.plot(range(0, n_epochs), [loss.detach().numpy() for loss in labeled_loss], label="labeled")
    plt.plot(range(0, n_epochs), [loss.detach().numpy() for loss in unlabeled_loss], label="unlabeled")
    plt.legend()
    plt.title(f"Loss: {weight_methods[i]}")
    plt.show()
  
    plt.plot(range(0, n_epochs), [acc for acc in labeled_acc], label="labeled")
    plt.plot(range(0, n_epochs), [acc for acc in unlabeled_acc], label="unlabeled")
    plt.legend()
    plt.title(f"Accuracy: {weight_methods[i]}")
    plt.show()
    
    min_unl_loss = np.argmin([i.item() for i in unlabeled_loss])
    print(f"Lowest unlabelled loss: {min(unlabeled_loss).detach().item()} at epoch {min_unl_loss}, train loss: {[i.item() for i in labeled_loss][min_unl_loss]}")
    
    print(f"Maximum acc: {max(unlabeled_acc)} at epoch {min_unl_loss}, train acc: {labeled_acc[min_unl_loss]}")

    print(f"Maximum f1: {max(unlabeled_f1)} at epoch {min_unl_loss}, train f1: {labeled_f1[min_unl_loss]}")
  
    i += 1

In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt

# make sure to run the train the GCN with preferred hyper-parameters and weights first

def draw(i):
    cls1color = '#00FFFF'
    cls2color = '#FF00FF'
    pos = {}
    colors = []
    for v in range(716):
        pos[v] = all_logits[i][v].numpy()
        cls = pos[v].argmax()
        colors.append(cls1color if cls else cls2color)
    ax.cla()
    ax.axis('off')
    ax.set_title('Epoch: %d' % i)
    nx.draw_networkx(G_cca.to_undirected(), node_color=colors,
            with_labels=False, node_size=5, ax=ax)

fig = plt.figure(dpi=150)
fig.clf()
ax = fig.subplots()
draw(0)  # draw the prediction of the first epoch
plt.close()

In [ ]:
G_test = G_cca

In [ ]:
remove = [node for node,degree in dict(G_test.degree()).items() if degree <= 2] # remove nodes with few edges for simplicity
G_test.remove_nodes_from(remove)

def draw(i):
    cls1color = '#C13002'
    cls2color = '#50006C'
    pos = {}
    colors = []
    for v in range(len(G_test.nodes)):
        pos[v] = all_logits[i][v].numpy()
        cls = pos[v].argmax()
        colors.append(cls1color if cls else cls2color)
    ax.cla()
    ax.axis('off')
    ax.set_title('Epoch: %d' % i)
    p=nx.spring_layout(G_test)
    nx.draw_networkx(G_test.to_undirected(), p, node_color=colors,
            with_labels=False, node_size=5, ax=ax, width=0.1)
    
fig = plt.figure(dpi=150)
fig.clf()
ax = fig.subplots()
draw(0)  # draw the prediction of the first epoch
plt.close()

In [ ]:
from matplotlib import rc
import matplotlib

matplotlib.rcParams['animation.embed_limit'] = 2**128
rc('animation', html='jshtml')
ani = animation.FuncAnimation(fig, draw, frames=len(all_logits), interval=200)

In [ ]:
# runs for a very long time
ani